In [78]:
import numpy as np

import scipy 
from scipy import optimize
from scipy.interpolate import interp1d

In [84]:
M200 = 1*10**8
C200 = 40
mnpr = 400
r0 = 0.001
dr = 1.001

file_name="_M_%.2e_c_%.2s_m_%s_dr_%s_r0_%s"%(M200,C200,mnpr,dr,r0)


h = 0.7
Omegam0 = 0.307
kpc = 3.0856*(10**16)
rhocrit = 277.5*(h**2)

eVJ = 1.602*10**(-25)
cc = 2.998*(10**5)
eVkg = eVJ/(cc**2)
gx = 2
hbar = 1.054*(10**(-40))/(kpc)
Msun = 2*(10**30)
fmax = ((mnpr*eVkg)**4)*gx/(2*(2*np.pi*hbar)**3)/Msun
GN = 4.30*(10**(-6))

f=lambda x:return np.log1p(x)-1+1/(1+x)

R200=((3*M200)/(4*np.pi*200*rhocrit))**(1.0/3)
rs=R200/C200
rho0=M200/(4*np.pi*rs**3*f(C200))


rhoNFW= lambda r,rho0,rs: rho0/((r/rs)*(1+r/rs)**2)


temp=lambda r,rho0,rs: rho0*rs*(-1/((r**2)*(1+r/rs)**2)-2/(r*rs*(1+r/rs)**3))

Psi=lambda r,rho0,rs: 4*GN*np.pi*rho0*(rs**3)*np.log1p(r/rs)/r



rmax1=10*scipy.optimize.fsolve(lambda x,rho0,rs: rhoNFW(x,rho0,rs)-Omegam0*rhocrit,r0, args=(rho0,rs))


kmax1=int((np.log10(rmax1/r0))/((np.log10(dr))))

In [85]:
def distr_function(Psi_arr,temp_arr,file_fe_name):
    k=len(temp_arr[0])
    fe=[[],[]]
    Ee1=Psi_arr[1][0]

    integrand=temp_arr[1][1:]/np.sqrt(Ee1-Psi_arr[1][1:])
    
    Int1=(Psi_arr[0][1]-Psi_arr[0][0])*(2*temp_arr[1][0])/(np.sqrt(-Psi_arr[1][1]+Psi_arr[1][0]))
    Int1=Int1+scipy.integrate.trapz(integrand, Psi_arr[0][1:])
    
    with open(file_fe_name, 'w') as file_fe:
        for i in range(1,k-1):
            Ee2=Psi_arr[1][i]
            integrand=[]
            integrand=temp_arr[1][i+1:]/np.sqrt(Ee2-Psi_arr[1][i+1:])

            Int2=(Psi_arr[0][i+1]-Psi_arr[0][i])*(2*temp_arr[1][i])/(np.sqrt(-Psi_arr[1][i+1]+Psi_arr[1][i]))
            Int2=Int2+scipy.integrate.trapz(integrand, Psi_arr[0][i+1:])
            fei=-(Int2-Int1)/(Ee2-Ee1)/(np.sqrt(8)*(np.pi)**2)
            fe[1].append(fei)
            fe[0].append(Ee1)
            print(np.log10(Ee1), np.log10(fei), np.log10(-Int1),   sep='   ', end='\n', file=file_fe)
            Int1=Int2
            Ee1=Ee2
            
    return np.array([fe[0], fe[1]])


def fe_tr(fe):
    
    fe[1,fe[1]>fmax]=fmax
    return fe
    
    
def rho_tr(fe,Psi_arr,file_rho_name):
    rho=[[],[]]
    with open (file_rho_name,'w') as file_rho:
    
        for m in range(len(fe[0])):

            Psix=fe[0][m]

            
            integrand2=[]
            integrand2=fe[1][m:]*np.sqrt(Psix-fe[0][m:])

            rho_temp=-(4)*np.sqrt(2)*np.pi*scipy.integrate.trapz(integrand2,fe[0][m:len(fe[0]):1])

            rho[1].append(rho_temp)

            rho[0].append(Psi_arr[0][m])
            print(np.log10(Psi_arr[0][m]), np.log10(rho_temp), sep='   ', end='\n', file=file_rho)
    
    
    
    return np.array(rho)

        

def temp_calc(rho):
    
    
    return np.array([rho[0,:-1],np.diff(rho[1])/np.diff(rho[0])])
        
def Psi_calc(rho,file_Psi_name):
    
    
    M0=np.pi*4/3*rho[1][0]*(rho[0][0])**3
    
    y=4*np.pi*rho[1]*rho[0]**2
    
    M_arr=np.append(np.array([M0]),M0+scipy.integrate.cumtrapz(y,rho[0]) )
        
    Psit=0
    Psi_arr=[[],[]]
    with open(file_Psi_name, 'w') as file_Psi:            
        for i in range(1,len(rho[0])):
            hx=(rho[0][-i]-rho[0][-i-1])
            Psit=Psit+GN*(M_arr[-i]/rho[0][-i]**2+M_arr[-i-1]/rho[0][-i-1]**2)*hx/2
            Psi_arr[1].append(Psit)
            Psi_arr[0].append(rho[0][-i-1])
            print(np.log10(rho[0][-i-1]), np.log10(Psit), sep='   ', end='\n', file=file_Psi)
    
   
    Psi_arr[0]=Psi_arr[0][::-1]
    Psi_arr[1]=Psi_arr[1][::-1]
    
    return np.array(Psi_arr)          

In [87]:
import time
t1=time.clock()
Psi_arr=[[],[]]
temp_arr=[[],[]]
rx=r0
for i in range(kmax1):
    
    Psi_arr[1].append(Psi(rx,rho0,rs))
    Psi_arr[0].append(rx)
    temp_arr[1].append(temp(rx,rho0,rs))
    temp_arr[0].append(rx)
    rx=rx*dr
Psi_arr=np.array(Psi_arr)

temp_arr=np.array(temp_arr)

for d in range(6):
    
    file_fe_name="fe_%s_%s.txt"%(file_name, str(d))
    fe = distr_function(Psi_arr,temp_arr,file_fe_name)
   
    fe_tr(fe)
        
    file_rho_name="rho_tr_%s_%s.txt"%(file_name, str(d))
    rho_arr = rho_tr(fe,Psi_arr,file_rho_name)
        
    file_Psi_name="Psi_%s_%s.txt"%(file_name,str(d))
    
    
    Psi_arr=Psi_calc(rho_arr,file_Psi_name)
    
    
    temp_arr=temp_calc(rho_arr)
    print(rho_arr[1,0])
    
t2=time.clock() 
print(t2-t1)

C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in log10


525841010.169
248832865.016
199102144.275
180798695.527
172529962.085
168406809.039
26.642301414409303
